In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Tasiilaq,GL,2020-06-24 02:34:33,65.61,-37.64,42.80,81,100,1.21
1,1,Ushuaia,AR,2020-06-24 02:34:20,-54.80,-68.30,33.80,86,90,18.34
2,2,Chokurdakh,RU,2020-06-24 02:36:51,70.63,147.92,76.69,26,49,8.08
3,3,Rikitea,PF,2020-06-24 02:36:51,-23.12,-134.97,69.94,53,84,9.53
4,4,Bluff,NZ,2020-06-24 02:36:51,-46.60,168.33,48.54,84,93,14.18


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5
                  )
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4) 
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:

# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=50, point_radius=4
                                )

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Chokurdakh,RU,2020-06-24 02:36:51,70.63,147.92,76.69,26,49,8.08
6,6,Dahanu,IN,2020-06-24 02:36:51,19.97,72.73,84.54,77,40,14.16
19,19,Albany,US,2020-06-24 02:35:03,42.60,-73.97,80.01,78,76,1.01
21,21,Broome,US,2020-06-24 02:36:53,42.25,-75.83,75.20,64,90,8.05
22,22,Mareeba,AU,2020-06-24 02:36:53,-17.00,145.43,77.00,61,75,18.34
24,24,Kapaa,US,2020-06-24 02:36:53,22.08,-159.32,82.40,69,1,11.41
25,25,Hithadhoo,MV,2020-06-24 02:36:53,-0.60,73.08,84.22,75,38,9.91
28,28,Avarua,CK,2020-06-24 02:34:34,-21.21,-159.78,78.80,74,57,5.82
30,30,Vaini,TO,2020-06-24 02:36:53,-21.20,-175.20,78.80,78,40,4.70
32,32,Hilo,US,2020-06-24 02:35:41,19.73,-155.09,82.40,58,90,8.05


In [11]:
preferred_cities_df.count()

City_ID       198
City          198
Country       198
Date          198
Lat           198
Lng           198
Max Temp      198
Humidity      198
Cloudiness    198
Wind Speed    198
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Chokurdakh,RU,76.69,70.63,147.92,
6,Dahanu,IN,84.54,19.97,72.73,
19,Albany,US,80.01,42.60,-73.97,
21,Broome,US,75.20,42.25,-75.83,
22,Mareeba,AU,77.00,-17.00,145.43,
24,Kapaa,US,82.40,22.08,-159.32,
25,Hithadhoo,MV,84.22,-0.60,73.08,
28,Avarua,CK,78.80,-21.21,-159.78,
30,Vaini,TO,78.80,-21.20,-175.20,
32,Hilo,US,82.40,19.73,-155.09,


In [31]:
# Set parameters to search for a hotel.
params = {
    "radius": 20000,
    "type": "lodging",
    "key": g_key
}

In [38]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 30000,
    "types": "lodging",
    "key": g_key,
    "location": "70.63, 147.92"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()


In [39]:
hotels


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [35]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Chokurdakh,RU,76.69,70.63,147.92,
6,Dahanu,IN,84.54,19.97,72.73,Visawa Beach Resort & Hotel Natural Picnic Spot
19,Albany,US,80.01,42.60,-73.97,The Inn at Deer Meadow
21,Broome,US,75.20,42.25,-75.83,Chenango Valley State Park
22,Mareeba,AU,77.00,-17.00,145.43,Trinity Plains Tourist Park


In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [45]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [46]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))